## How to Use

1. Run the job with `-L <rate>`.
2. Create `rays.csv` and `bags.csv by running the following commands in the log directory:
```
r2t2/scripts/parse_worker_info.py -i . -t RAG -o rays.csv
r2t2/scripts/parse_worker_info.py -i . -t BAG -o bags.csv
```

**Make sure** you have `numpy`, `matplotlib` and `pandas` installed.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display

plt.rcParams['figure.figsize'] = 12, 10
plt.rcParams.update({'font.size': 16})
pd.options.display.max_rows = 20

def show_more(d, lines):
    save = pd.options.display.max_rows
    pd.options.display.max_rows = lines
    display(d)
    pd.options.display.max_rows = save
    
def show_all(d):
    return show_more(d, len(d))

def read_data(rays, bags):
    ray_data = pd.read_csv(rays, dtype={'bag': object})
    bag_data = pd.read_csv(bags)
    
    min_timestamp_ray = min(ray_data.timestamp)
    min_timestamp_bag = min(bag_data.timestamp)
    min_timestamp = min(min_timestamp_ray, min_timestamp_bag)
    
    ray_data['timestamp_ms'] = ray_data.timestamp - min_timestamp
    bag_data['timestamp_ms'] = bag_data.timestamp - min_timestamp
    
    del ray_data['timestamp']
    del bag_data['timestamp']
    
    return ray_data, bag_data

def show_path(data, path_id):
    p = (data[(data.pathId == path_id) & (data.shadowRay == 0)]
        .loc[:, ['hop', 'workerId', 'treeletId', 'timestamp_ms', 'action', 'bag']]
        .sort_values(by=['hop', 'timestamp_ms'])).reset_index(drop=True)
    
    def highlighter(x):
        return ['background-color: rgba(255, 255, 255, 0.1)' if (x.hop % 2 == 0) else '' for y in x]
    
    q = p.style.apply(highlighter, axis=1)
    show_more(q, len(p))
    return q

def show_longest_paths(data):
    return (data[data.action.isin(['Generated', 'Finished'])]
            .groupby('pathId')
            .agg({'timestamp_ms': 'max', 'hop': 'max'})
            .sort_values('timestamp_ms', ascending=False)
            .head(20))

In [ ]:
rays, bags = read_data("/data/cloudrt/logs/2019-12-30/killeroo-simple/rays.csv",
                       "/data/cloudrt/logs/2019-12-30/killeroo-simple/bags.csv")

In [ ]:
show_longest_paths(rays)